In [3]:
import os
import glob
import base64
import pandas as pd
from openai import OpenAI
from tqdm import tqdm  # 引入tqdm来显示进度条a
import time

# --- 1. 配置 ---

client = OpenAI(
    api_key="sk-34e1c3af4b354d7fb9828e437b8895d4",  # 替换为你的 API Key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 数据集路径
IMAGE_DIR_1 = "data/HAM10000/HAM10000_images_part_1"
IMAGE_DIR_2 = "data/HAM10000/HAM10000_images_part_2"
METADATA_FILE = "data/HAM10000/HAM10000_metadata.csv"

# 结果保存路径
RESULTS_FILE = "classification_results.csv"

# 使用方案二的提示词，因为它在效果和简洁性之间取得了很好的平衡
categories = ["akiec", "bcc", "bkl", "df", "mel", "nv", "vasc"]
categories_str = ", ".join(categories)
PROMPT_TEXT = f"""
你是一名顶级的皮肤病理学AI诊断助手。请遵循以下步骤对提供的皮肤镜图像进行分析和分类。

**第一步：特征分析 (内心思考，不要输出)**
1.  **对称性 (Symmetry)**: 病变形状是否对称？
2.  **边缘 (Border)**: 边缘是清晰、规则，还是模糊、不规则、有切迹？
3.  **颜色 (Color)**: 颜色是单一均匀，还是包含多种颜色（如棕、黑、红、蓝、白）？颜色分布是否均匀？
4.  **结构 (Structures)**: 是否能观察到特定的皮肤镜结构？例如，色素网络、点状血管、树枝状血管、蓝白幕、乳头状结构等。
5.  **整体评估**: 综合以上特征，病变给人的整体感觉是良性的（有序、规则）还是恶性的（混乱、不规则）？

**第二步：分类判断**
基于以上分析，将图像归类到以下七个类别之一：{categories_str}。
- `mel`: 高度怀疑恶性，通常符合ABCDE法则（不对称、边缘不规则、颜色杂乱、直径大、有变化）。
- `bcc`: 可见树枝状血管、溃疡或珍珠光泽边缘。
- `nv`: 结构和颜色对称、规则。
- `bkl`: 具有“粘贴感”，可能看到粉刺样开口或脑回状结构。
- `akiec`: 表面有鳞屑，背景常为红斑。
- `df`: 中央常为瘢痕样白色区域，周边有细小的色素网络。
- `vasc`: 主要由红色或紫色的血管结构构成。

**第三步：输出结果**
请只输出你最终判断的类别缩写，不要包含任何分析过程、解释或额外文字。
"""



In [ ]:
# grok4 cot
prompt = f"""
你是一名国际顶级的皮肤病理学AI诊断专家。现在你正在分析一张皮肤镜图像，
该图像已经被确诊为 **'{diagnosis_fullname}'**。

你的任务是：
**基于图像的视觉证据，通过思维链逐步推理，详细解释为什么该图像符合此诊断。**
请用专业、逻辑清晰的方式，一步步系统性地分析支持该诊断的关键视觉特征。让我们一步一步思考。

请遵循以下思维链分析结构，每步基于前一步的观察进行推理：

1. **观察总体结构与形态**  
   - 首先，观察病变的整体形态，包括对称性、结构组织和大致大小比例。  
   - 推理：基于这些观察，该病灶呈现规则或不规则形态？这如何初步指向可能的诊断方向？

2. **分析边缘特征**  
   - 接下来，检查病变的边界：是否清晰、平滑、模糊或有放射状延伸？  
   - 推理：结合总体形态，这些边界特征如何提示病变是良性还是恶性？这步观察如何强化或调整上一步的初步判断？

3. **评估颜色与分布模式**  
   - 然后，识别病变中的颜色种类（如棕色、黑色、红色、蓝灰色、白色等）。  
   - 分析颜色的分布：是否均匀？是否存在局部色素集中或色调突变？  
   - 推理：这些颜色特征与前两步的形态和边缘观察相结合，如何进一步支持特定诊断？颜色分布提供了哪些额外证据？

4. **识别关键皮肤镜结构并得出诊断依据**  
   - 最后，基于前三步的综合观察，根据具体诊断类别，识别最具代表性的皮肤镜结构与模式，并解释其意义：  

     - **黑色素瘤 (mel, Melanoma)**  
       特征：明显的不对称性、不规则边缘、颜色多样性（棕、黑、蓝、白、红等），非典型色素网络、蓝白幕、放射状线条、负网状结构、不对称的小点或条纹、局部回避区等恶性特征。  
       推理：这些结构如何与先前步骤匹配，确认恶性潜力？

     - **基底细胞癌 (bcc, Basal Cell Carcinoma)**  
       特征：树枝状血管、蓝灰色卵圆巢、光滑珠光边缘、溃疡或结痂区域、车轮辐射状结构、白色条纹或亮点。  
       推理：这些血管和边缘特征如何整合前述观察，指向BCC？

     - **黑色素细胞痣 (nv, Melanocytic Nevus)**  
       特征：整体对称、规则的色素网络、均匀的棕色色调、清晰边界、可见规则点状或球状结构、均匀分布的色素网格。  
       推理：规则性和均匀性如何确认良性痣的诊断？

     - **脂溢性角化病 (bkl, Benign Keratosis)**  
       特征：粉刺样开口、脑回状（丘脑状）结构、粘贴感外观、白色假网状结构、角质栓、黑点或伪毛囊口。  
       推理：这些表面结构如何与颜色和形态匹配BKL？

     - **光化性角化病 (akiec, Actinic Keratosis)**  
       特征：红白交错的表面、毛细血管扩张、鳞屑、角质过度增生、淡棕或红色调，可能可见“草地样”或“红白斑块状”结构。  
       推理：表面纹理和颜色变化如何支持AKIEC诊断？

     - **皮肤纤维瘤 (df, Dermatofibroma)**  
       特征：中心棕色区伴周围淡色晕、放射状色素结构、中心瘢痕样白区、周边色素网络逐渐消退、轻微凹陷。  
       推理：中心-周边模式如何整合所有观察？

     - **血管性病变 (vasc, Vascular Lesion)**  
       特征：均匀的红色至紫色区域、清晰可见的血管结构、点状或线状血管、湖状血管样分布、整体对称。  
       推理：血管主导特征如何确认血管性病变？

请用简洁、条理清晰的专家语言作答，每步结束时明确说明推理逻辑。
你的回答应聚焦于**图像可见特征与诊断逻辑的逐步对应关系**，
避免空泛描述或直接跳到结论，最终基于所有步骤得出诊断确认，你的诊断过程将是宝贵的学习材料
"""


In [4]:
# --- 2. 图像分类函数 ---

def classify_image(image_path):
    """对单个图像进行分类并返回结果"""
    try:
        with open(image_path, 'rb') as img_file:
            img_base64 = base64.b64encode(img_file.read()).decode('utf-8')
        data_url = f"data:image/jpeg;base64,{img_base64}"
        
        completion = client.chat.completions.create(
            model="qwen2.5-vl-7b-instruct",
            messages=[{
                "role": "user",
                "content": [
                    {"type": "text", "text": PROMPT_TEXT},
                    {"type": "image_url", "image_url": {"url": data_url}}
                ]
            }]
        )
        # 清理和规范化输出
        prediction = completion.choices[0].message.content.strip().lower()
        # 确保输出是我们定义的类别之一，如果不是则标记为未知
        if prediction not in categories:
            print(f"警告：模型返回了未知类别 '{prediction}'，将标记为 'unknown'。")
            return "unknown"
        return prediction
    except Exception as e:
        print(f"处理图像 {os.path.basename(image_path)} 时发生API错误: {e}")
        # 返回 'error' 标记，以便后续处理
        return "error"



In [ ]:
# --- 3. 主执行逻辑 ---
print("步骤 1: 加载元数据和图像列表...")

# 加载元数据
metadata_df = pd.read_csv(METADATA_FILE)

# 获取所有图像路径
all_image_paths = glob.glob(os.path.join(IMAGE_DIR_1, '*.jpg')) + \
                  glob.glob(os.path.join(IMAGE_DIR_2, '*.jpg'))

# 创建一个 image_id 到 path 的映射，方便查找
image_path_map = {os.path.splitext(os.path.basename(p))[0]: p for p in all_image_paths}

# 过滤元数据，只保留我们拥有图像文件的记录
image_ids_we_have = list(image_path_map.keys())
target_df = metadata_df[metadata_df['image_id'].isin(image_ids_we_have)].copy()

print(f"总共找到 {len(all_image_paths)} 张图像文件。")
print(f"元数据中与之对应的记录有 {len(target_df)} 条。")
print("\n步骤 2: 开始进行图像分类 (支持断点续传)...")

# 加载已有的结果以实现断点续传
completed_image_ids = set()
if os.path.exists(RESULTS_FILE):
    results_df = pd.read_csv(RESULTS_FILE)
    completed_image_ids = set(results_df['image_id'])
    print(f"已在结果文件中找到 {len(completed_image_ids)} 条记录，将跳过这些图像。")
# 打开结果文件准备追加写入
with open(RESULTS_FILE, 'a', newline='', encoding='utf-8') as f:
    # 如果是新文件，写入表头
    if not completed_image_ids:
        f.write("image_id,prediction\n")
    # 使用 tqdm 创建进度条
    for index, row in tqdm(target_df.iterrows(), total=target_df.shape[0], desc="分类进度"):
        image_id = row['image_id']
        
        # 断点续传逻辑：如果已处理，则跳过
        if image_id in completed_image_ids:
            continue
        
        image_path = image_path_map.get(image_id)
        if not image_path:
            continue
        # 调用API进行分类
        prediction = classify_image(image_path)
        
        # 立刻将结果写入文件
        f.write(f"{image_id},{prediction}\n")
        
        # API调用频率控制，避免过于频繁导致被限制
        time.sleep(0.5) 
print("\n所有图像分类完成！结果已保存至 'classification_results.csv'。")


步骤 1: 加载元数据和图像列表...
总共找到 10015 张图像文件。
元数据中与之对应的记录有 10015 条。

步骤 2: 开始进行图像分类 (支持断点续传)...
已在结果文件中找到 8921 条记录，将跳过这些图像。


分类进度:  95%|█████████▍| 9471/10015 [1:26:57<49:28,  5.46s/it]  

处理图像 ISIC_0032554.jpg 时发生API错误: Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-0fc219ba-86a8-4448-88ee-698bb68654f5', 'request_id': '0fc219ba-86a8-4448-88ee-698bb68654f5'}


分类进度:  95%|█████████▌| 9520/10015 [1:35:01<1:32:11, 11.18s/it]

In [ ]:
def retry_error_predictions():
    print("步骤 2b: 重新尝试之前标记为 error 的图像...")

    # 若结果文件不存在则直接返回
    if not os.path.exists(RESULTS_FILE):
        print("未找到结果文件，跳过重试。")
        return

    # 读取已有预测结果
    predictions_df = pd.read_csv(RESULTS_FILE)

    # 找到所有 error 的记录
    error_df = predictions_df[predictions_df['prediction'] == 'error'].copy()
    if error_df.empty:
        print("没有需要重试的 error 记录。")
        return

    # 重建 image_id -> path 映射（两文件夹）
    all_image_paths = glob.glob(os.path.join(IMAGE_DIR_1, '*.jpg')) + glob.glob(os.path.join(IMAGE_DIR_2, '*.jpg'))
    image_path_map = {os.path.splitext(os.path.basename(p))[0]: p for p in all_image_paths}

    print(f"需要重试的图像数: {len(error_df)}")

    # 对每个 error 记录重试
    for image_id in tqdm(error_df['image_id'], desc="错误重试进度", total=len(error_df)):
        image_path = image_path_map.get(image_id)
        if not image_path:
            print(f"未找到图像文件: {image_id}，保持为 error。")
            continue

        # 重新分类
        new_pred = classify_image(image_path)
        # 更新到 DataFrame
        predictions_df.loc[predictions_df['image_id'] == image_id, 'prediction'] = new_pred

        # 控制调用频率
        time.sleep(1)

    # 覆盖写回结果文件
    predictions_df.to_csv(RESULTS_FILE, index=False)
    print("错误记录已重试并更新保存至 classification_results.csv。")


# 运行重试函数（也可在需要时单独运行此行）
retry_error_predictions()

In [ ]:
# --- 4. 结果分析 ---
print("\n步骤 3: 分析分类结果...")

# 读取完整的真实标签和预测结果
ground_truth_df = pd.read_csv(METADATA_FILE)[['image_id', 'dx']]
predictions_df = pd.read_csv(RESULTS_FILE)

# 合并数据
comparison_df = pd.merge(ground_truth_df, predictions_df, on='image_id')

# 过滤掉API调用失败的记录
comparison_df = comparison_df[comparison_df['prediction'] != 'error']

# 计算总体准确率
correct_predictions = (comparison_df['dx'] == comparison_df['prediction']).sum()
total_predictions = len(comparison_df)
overall_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0

print("\n--- 分类结果报告 ---")
print(f"总计分析图像数: {total_predictions}")
print(f"模型预测正确数: {correct_predictions}")
print(f"整体准确率: {overall_accuracy:.2%}")

# 使用 pd.crosstab 生成分类混淆矩阵
print("\n分类混淆矩阵 (行: 真实类别, 列: 预测类别):")
confusion_matrix = pd.crosstab(comparison_df['dx'], comparison_df['prediction'])
print(confusion_matrix)

# 计算每个类别的准确率
print("\n各类别详细准确率:")
for category in categories:
    cat_truth = comparison_df[comparison_df['dx'] == category]
    if not cat_truth.empty:
        cat_correct = (cat_truth['dx'] == cat_truth['prediction']).sum()
        cat_accuracy = cat_correct / len(cat_truth)
        print(f"- {category:>5s}: {cat_accuracy:.2%} ({cat_correct}/{len(cat_truth)})")
    else:
        print(f"- {category:>5s}: 数据不足，无该类别样本。")
        
print("\n--- 报告结束 ---")